In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.llms import HuggingFacePipeline
import torch

In [9]:
import pandas as pd
from langchain_community.llms import Ollama
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts.chat import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

In [4]:
# φόρτωση του fine-tuned Intent Classifier
from transformers import AutoModelForSequenceClassification, AutoTokenizer

classifier_model = AutoModelForSequenceClassification.from_pretrained("evelynkol/intent-classifier")
classifier_tokenizer = AutoTokenizer.from_pretrained("evelynkol/intent-classifier")

config.json:   0%|          | 0.00/769 [00:00<?, ?B/s]

C:\Users\evage\anaconda3\envs\clean_env\lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\evage\.cache\huggingface\hub\models--evelynkol--intent-classifier. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [6]:
# Load the locally installed LLaMA model using Ollama
llama2 = Ollama(model="llama3.2:1b")

In [22]:
from langchain_community.llms import Ollama
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

chat_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a knowledgeable medical assistant chatbot. Provide accurate and helpful answers to medical questions. Conversation so far:\n{history}"),
        ("human", "{input}"),
    ]
)

# χρηση Llama2 μοντέλου 
llama2 = Ollama(model="llama3.2:1b")

# δημιουργία memory συνομολίας 
memory = ConversationBufferMemory()

# chain συζήτησης 
med_question_chain = ConversationChain(
    llm=llama2,
    prompt=chat_prompt,
    memory=memory
)

In [25]:
# Function to classify intent using the intent classifier
def classify_intent(user_input):
    inputs = classifier_tokenizer(user_input, return_tensors="pt", truncation=True, padding=True)
    outputs = classifier_model(**inputs)
    intent_id = torch.argmax(outputs.logits, dim=1).item()
    return intent_id  # 1 for 'med_question', 0 for other intents like 'symptoms'


# Function to handle medical questions
def handle_med_question(user_input):
    # Use the medical question chain for generating a response
    response = med_question_chain.run(input=user_input)
    return response


# Main function to handle user input
def chatbot(user_input):
    # Step 1: Classify the intent
    intent = classify_intent(user_input)
    print(intent)

    # Step 2: Handle the intent
    if intent == 1:  # Medical question
        response = handle_med_question(user_input)
    else:
        response = "I'm sorry, I can't understand your input. Please describe your symptoms or ask a medical question."
    
    return response


# Example usage
user_input = "What are the side effects of ibuprofen?"
response = chatbot(user_input)
print(f"Chatbot: {response}")

1
Chatbot: I can provide more information on the side effects of ibuprofen. In addition to the gastrointestinal issues you mentioned earlier, such as stomach upset, nausea, and vomiting, some other potential side effects of ibuprofen include:

* Kidney damage or dysfunction: Ibuprofen may increase the risk of kidney damage or dysfunction, especially in individuals with pre-existing kidney problems.
* Liver damage or dysfunction: There have been reports of liver damage and dysfunction associated with long-term use of ibuprofen.
* Allergic reactions: As mentioned earlier, allergic reactions can include hives, itching, difficulty breathing, and other symptoms.
* Increased risk of heart attack, stroke, or other cardiovascular events: Ibuprofen may increase the risk of heart attack, stroke, and other cardiovascular events in certain individuals.

It's also worth noting that ibuprofen can interact with certain medications, such as:

* Blood thinners (e.g., warfarin)
* Diabetes medications
* 